In [40]:
# -*- coding: utf-8 -*-
"""
Created on 21 Aug 2014
Functions and classes for convolutional and turbo codes.
Author: Venkat Venkatesan
from https://github.com/venkat0791/codec
"""
import math
import numpy as np
from AWGN import _AWGN
ch=_AWGN()

In [41]:
class Conv(object):
  INF = 1e6

  def __init__(self, back_poly, fwd_polys):

    # Number of bits in encoder state.
    self.mem_len = math.floor(math.log(back_poly) / math.log(2))

    # Encoder state space (integers in the range [0, 2 ** mem_len)).
    self.state_space = tuple(n for n in range(1 << self.mem_len))

    # Number of encoder output bits per input bit.
    self.n_out = len(fwd_polys)

    # MSB of next encoder state, given current state and input bit.
    self.next_state_msb = tuple(tuple(self.bitxor(back_poly & ((b << self.mem_len) + s))for s in self.state_space) for b in (0, 1))

    # Encoder output bits, given current state and input bit.
    self.out_bits = tuple(tuple(tuple(self.bitxor(p & ((self.next_state_msb[b][s] << self.mem_len) + s))for p in fwd_polys) for s in self.state_space) for b in (0, 1))
    
    # Next encoder state, given current state and input bit.
    self.next_state = tuple(tuple((self.next_state_msb[b][s] << (self.mem_len - 1)) + (s >> 1)for s in self.state_space) for b in (0, 1))

    return

In [42]:
class Conv(Conv):

  @staticmethod
  def bitxor(num):
    '''
    Returns the XOR of the bits in the binary representation of the
    nonnegative integer num.
    '''

    count_of_ones = 0
    while num > 0:
      count_of_ones += num & 1
      num >>= 1

    return count_of_ones % 2

  @staticmethod
  def maxstar(eggs, spam, max_log=False):
    '''
    Returns log(exp(eggs) + exp(spam)) if not max_log, and max(eggs, spam)
    otherwise.
    '''
    return max(eggs, spam) + (0 if max_log else math.log(1 + math.exp(-abs(spam - eggs))))

In [43]:
class Conv(Conv):

  def encode(self, info_bits):

    info_bits = np.asarray(info_bits).ravel()
    n_info_bits = info_bits.size

    code_bits, enc_state = -np.ones(
    self.n_out * (n_info_bits + self.mem_len), dtype=int), 0
    
    for k in range(n_info_bits + self.mem_len):
      in_bit = (info_bits[k] if k < n_info_bits else self.next_state_msb[0][enc_state])
      code_bits[self.n_out * k : self.n_out * (k + 1)] = (self.out_bits[in_bit][enc_state])
      enc_state = self.next_state[in_bit][enc_state]

    return code_bits

In [44]:
class Conv(Conv):

  def _branch_metrics(self, out_bit_llrs, pre_in_bit_llr=0):

    gamma_val = ([pre_in_bit_llr / 2 for s in self.state_space],[-pre_in_bit_llr / 2 for s in self.state_space])
    for enc_state in self.state_space:
      for bit0, bit1, val in zip(self.out_bits[0][enc_state],self.out_bits[1][enc_state],out_bit_llrs):
        gamma_val[0][enc_state] += val / 2 if bit0 == 0 else -val / 2
        gamma_val[1][enc_state] += val / 2 if bit1 == 0 else -val / 2

    return gamma_val

  def _update_path_metrics(self, out_bit_llrs, path_metrics, best_bit):

    gamma_val = self._branch_metrics(out_bit_llrs)

    pmn = path_metrics[:]
    for enc_state in self.state_space:
      cpm0 = gamma_val[0][enc_state] + pmn[self.next_state[0][enc_state]]
      cpm1 = gamma_val[1][enc_state] + pmn[self.next_state[1][enc_state]]
      path_metrics[enc_state], best_bit[enc_state] = ((cpm0, 0) if cpm0 >= cpm1 else (cpm1, 1))

    return

In [45]:
class Conv(Conv):
  def decode_viterbi(self, code_bit_llrs):

    code_bit_llrs = np.asarray(code_bit_llrs).ravel()
    n_in_bits = int(code_bit_llrs.size / self.n_out)
    n_info_bits = n_in_bits - self.mem_len

    # Path metric for each state at time n_in_bits.
    path_metrics = [(0 if s == 0 else -Conv.INF) for s in self.state_space]

    # Best input bit in each state at times 0 to n_in_bits - 1.
    best_bit = [[-1 for s in self.state_space] for k in range(n_in_bits)]

    # Start at time n_in_bits - 1 and work backward to time 0, finding
    # path metric and best input bit for each state at each time.
    for k in range(n_in_bits - 1, -1, -1):
      self._update_path_metrics(
      code_bit_llrs[self.n_out * k : self.n_out * (k + 1)],path_metrics, best_bit[k])

    # Decode by starting in state 0 at time 0 and tracing path
    # corresponding to best input bits.
    info_bits_hat, enc_state = -np.ones(n_info_bits, dtype=int), 0
    for k in range(n_info_bits):
      info_bits_hat[k] = best_bit[k][enc_state]
      enc_state = self.next_state[info_bits_hat[k]][enc_state]

    return info_bits_hat

  def _update_alpha(self,out_bit_llrs,pre_in_bit_llr,alpha_val,alpha_val_next,max_log):

    gamma_val = self._branch_metrics(out_bit_llrs, pre_in_bit_llr)

    for enc_state in self.state_space:
      alpha_val_next[self.next_state[0][enc_state]] = self.maxstar(alpha_val_next[self.next_state[0][enc_state]],alpha_val[enc_state] + gamma_val[0][enc_state],max_log)
      alpha_val_next[self.next_state[1][enc_state]] = self.maxstar(alpha_val_next[self.next_state[1][enc_state]],alpha_val[enc_state] + gamma_val[1][enc_state],max_log)

    return


In [46]:
class Conv(Conv):

  def _update_beta_tail(self, out_bit_llrs, beta_val, max_log):

    gamma_val = self._branch_metrics(out_bit_llrs, 0)

    bvn = beta_val[:]
    for enc_state in self.state_space:
      beta_val[enc_state] = self.maxstar(gamma_val[0][enc_state] + bvn[self.next_state[0][enc_state]],gamma_val[1][enc_state] + bvn[self.next_state[1][enc_state]],max_log)
    return

  def _update_beta(self,out_bit_llrs,pre_in_bit_llr,alpha_val,beta_val,max_log):

    gamma_val = self._branch_metrics(out_bit_llrs, pre_in_bit_llr)

    met0 = -Conv.INF
    met1 = -Conv.INF
    bvn = beta_val[:]
    for enc_state in self.state_space:
      beta_val[enc_state] = self.maxstar(gamma_val[0][enc_state] + bvn[self.next_state[0][enc_state]],gamma_val[1][enc_state] + bvn[self.next_state[1][enc_state]],max_log)
      met0 = self.maxstar(alpha_val[enc_state] + gamma_val[0][enc_state]+ bvn[self.next_state[0][enc_state]],met0,max_log)
      met1 = self.maxstar(alpha_val[enc_state] + gamma_val[1][enc_state]+ bvn[self.next_state[1][enc_state]],met1,max_log)
    return met0 - met1

In [47]:
class Conv(Conv):
  def decode_bcjr(self,code_bit_llrs,pre_info_bit_llrs=None,max_log=False):

    code_bit_llrs = np.asarray(code_bit_llrs).ravel()
    n_in_bits = int(code_bit_llrs.size / self.n_out)
    n_info_bits = n_in_bits - self.mem_len

    if pre_info_bit_llrs is None:
      pre_info_bit_llrs = np.zeros(n_info_bits)
    else:
      pre_info_bit_llrs = np.asarray(pre_info_bit_llrs).ravel()

    # FORWARD PASS: Recursively compute alpha values for all states at
    # all times from 1 to n_info_bits - 1, working forward from time 0.
    alpha = [[(0 if s == 0 and k == 0 else -Conv.INF)for s in self.state_space] for k in range(n_info_bits)]
    for k in range(n_info_bits - 1):
      out_bit_llrs = code_bit_llrs[self.n_out * k : self.n_out * (k + 1)]
      self._update_alpha(out_bit_llrs, pre_info_bit_llrs[k],alpha[k], alpha[k + 1], max_log)

    # BACKWARD PASS (TAIL): Recursively compute beta values for all
    # states at time n_info_bits, working backward from time n_in_bits.
    beta = [(0 if s == 0 else -Conv.INF) for s in self.state_space]
    for k in range(n_in_bits - 1, n_info_bits - 1, -1):
      out_bit_llrs = code_bit_llrs[self.n_out * k : self.n_out * (k + 1)]
      self._update_beta_tail(out_bit_llrs, beta, max_log)

    # BACKWARD PASS: Recursively compute beta values for all states at
    # each time k from 0 to n_info_bits - 1, working backward from time
    # n_info_bits, and also obtaining the post-decoding LLR for the info
    # bit at each time.
    post_info_bit_llrs = np.zeros_like(pre_info_bit_llrs)
    for k in range(n_info_bits - 1, - 1, -1):
      out_bit_llrs = code_bit_llrs[self.n_out * k : self.n_out * (k + 1)]
      post_info_bit_llrs[k] = self._update_beta(out_bit_llrs, pre_info_bit_llrs[k],alpha[k], beta, max_log)

    return post_info_bit_llrs

In [48]:
class Turbo(object):

  def __init__(self, back_poly, parity_polys):

    # Encoder and decoder for constituent RSC code
    self.rsc = Conv(back_poly, [back_poly] + parity_polys)

    # Number of output bits per input bit and number of tail bits
    # per input block for the turbo code
    self.n_out = self.rsc.n_out + (self.rsc.n_out - 1)
    self.n_tail_bits = self.rsc.n_out * self.rsc.mem_len * 2

    # Turbo interleaver and deinterleaver
    self.turbo_int, self.turbo_deint = [], []

    return

  @staticmethod
  def turbo_int_3gpp2(n_bits):

    # Look-up table
    int_table = (
      (1, 1, 3, 5, 1, 5, 1, 5,
      3, 5, 3, 5, 3, 5, 5, 1,
      3, 5, 3, 5, 3, 5, 5, 5,
      1, 5, 1, 5, 3, 5, 5, 3),
      (5, 15, 5, 15, 1, 9, 9, 15,
      13, 15, 7, 11, 15, 3, 15, 5,
      13, 15, 9, 3, 1, 3, 15, 1,
      13, 1, 9, 15, 11, 3, 15, 5),
      (27, 3, 1, 15, 13, 17, 23, 13,
      9, 3, 15, 3, 13, 1, 13, 29,
      21, 19, 1, 3, 29, 17, 25, 29,
      9, 13, 23, 13, 13, 1, 13, 13),
      (3, 27, 15, 13, 29, 5, 1, 31,
      3, 9, 15, 31, 17, 5, 39, 1,
      19, 27, 15, 13, 45, 5, 33, 15,
      13, 9, 15, 31, 17, 5, 15, 33),
      (15, 127, 89, 1, 31, 15, 61, 47,
      127, 17, 119, 15, 57, 123, 95, 5,
      85, 17, 55, 57, 15, 41, 93, 87,
      63, 15, 13, 15, 81, 57, 31, 69),
      (3, 1, 5, 83, 19, 179, 19, 99,
      23, 1, 3, 13, 13, 3, 17, 1,
      63, 131, 17, 131, 211, 173, 231, 171,
      23, 147, 243, 213, 189, 51, 15, 67),
      (13, 335, 87, 15, 15, 1, 333, 11,
      13, 1, 121, 155, 1, 175, 421, 5,
      509, 215, 47, 425, 295, 229, 427, 83,
      409, 387, 193, 57, 501, 313, 489, 391),
      (1, 349, 303, 721, 973, 703, 761, 327,
      453, 95, 241, 187, 497, 909, 769, 349,
      71, 557, 197, 499, 409, 259, 335, 253,
      677, 717, 313, 757, 189, 15, 75, 163))

    # Results of bit-reversing 0,1,...,31
    bit_rev32 = (
      0, 16, 8, 24, 4, 20, 12, 28,
      2, 18, 10, 26, 6, 22, 14, 30,
      1, 17, 9, 25, 5, 21, 13, 29,
      3, 19, 11, 27, 7, 23, 15, 31)

    # Integer n such that 3 <= n <= 10
    interleaver_param = math.ceil(math.log(n_bits) / math.log(2)) - 5

    # Mask used to extract n LSBs of an integer
    ctr_mask = (1 << interleaver_param) - 1

    # Generate interleaver and deinterleaver
    turbo_int = [-1 for n in range(n_bits)]
    turbo_deint = [-1 for n in range(n_bits)]
    in_addr = 0
    for ctr in range(1 << (interleaver_param + 5)):
      msb, lsb = ctr >> 5, ctr & 31
      new_val = (msb + 1) & ctr_mask
      lut_out = int_table[interleaver_param - 3][lsb]
      out_addr = ((bit_rev32[lsb] << interleaver_param)+((new_val * lut_out) & ctr_mask))
      if out_addr < n_bits:
        turbo_int[in_addr], turbo_deint[out_addr] = out_addr, in_addr
        in_addr += 1

    return turbo_int, turbo_deint

  @staticmethod
  def interleave(n_info_bits):
    turbo_int=np.arange(n_info_bits)
    np.random.shuffle(turbo_int)
    turbo_deint=np.argsort(turbo_int)
    return turbo_int,turbo_deint

In [49]:
class Turbo(Turbo):

  def encode(self, info_bits):

    info_bits = np.asarray(info_bits).ravel()
    n_info_bits = info_bits.size

    if n_info_bits != len(self.turbo_int):
      self.turbo_int, self.turbo_deint = self.interleave(n_info_bits)

    # Get code bits from each encoder.
    ctop = self.rsc.encode(info_bits)
    cbot = self.rsc.encode(info_bits[self.turbo_int])

    # Assemble code bits from both encoders.
    code_bits, pos = -np.ones(self.n_out * n_info_bits + self.n_tail_bits, dtype=int), 0

    for k in range(n_info_bits):
      code_bits[pos : pos + self.rsc.n_out] = ctop[self.rsc.n_out * k : self.rsc.n_out * (k + 1)]
      pos += self.rsc.n_out
      code_bits[pos : pos + self.rsc.n_out - 1] = cbot[self.rsc.n_out * k + 1 : self.rsc.n_out * (k + 1)]
      pos += self.rsc.n_out - 1
    code_bits[pos : pos + self.rsc.n_out * self.rsc.mem_len] = ctop[self.rsc.n_out * n_info_bits :]
    code_bits[pos + self.rsc.n_out * self.rsc.mem_len :] = cbot[self.rsc.n_out * n_info_bits :]

    return code_bits

In [50]:
class Turbo(Turbo):
  
  def decode(self, code_bit_llrs, n_turbo_iters, max_log=False):

    code_bit_llrs = np.asarray(code_bit_llrs).ravel()
    n_info_bits = int((code_bit_llrs.size - self.n_tail_bits) / self.n_out)

    if n_info_bits != len(self.turbo_int):
      self.turbo_int, self.turbo_deint = self.interleave(n_info_bits)

    # Systematic bit LLRs for each decoder
    sys_llrs_top = code_bit_llrs[0 : self.n_out * n_info_bits : self.n_out]
    sys_llrs_bot = sys_llrs_top[self.turbo_int]

    # Code bit LLRs for each decoder
    ctop_llrs = np.zeros(self.rsc.n_out * (n_info_bits + self.rsc.mem_len))
    cbot_llrs = np.zeros(self.rsc.n_out * (n_info_bits + self.rsc.mem_len))
    pos = 0
    for k in range(n_info_bits):
      num = self.rsc.n_out * k
      ctop_llrs[num] = sys_llrs_top[k]
      cbot_llrs[num] = sys_llrs_bot[k]
      pos += 1
      ctop_llrs[num + 1 : num + self.rsc.n_out] = code_bit_llrs[pos : pos + self.rsc.n_out - 1]
      pos += self.rsc.n_out - 1
      cbot_llrs[num + 1 : num + self.rsc.n_out] = code_bit_llrs[pos : pos + self.rsc.n_out - 1]
      pos += self.rsc.n_out - 1
      
    ctop_llrs[self.rsc.n_out * n_info_bits :] = code_bit_llrs[pos : pos + self.rsc.n_out * self.rsc.mem_len]
    cbot_llrs[self.rsc.n_out * n_info_bits :] = code_bit_llrs[pos + self.rsc.n_out * self.rsc.mem_len :]

    #puncturing
    ctop_llrs[1::2]=0
    cbot_llrs[0::2]=0

    # Main loop for turbo iterations
    ipre_llrs, ipost_llrs = np.zeros(n_info_bits), np.zeros(n_info_bits)
    for _ in range(n_turbo_iters):
      ipost_llrs[:] = self.rsc.decode_bcjr(ctop_llrs, ipre_llrs, max_log)
      ipre_llrs[:] = (ipost_llrs[self.turbo_int]- ipre_llrs[self.turbo_int]- sys_llrs_top[self.turbo_int])
      ipost_llrs[:] = self.rsc.decode_bcjr(cbot_llrs, ipre_llrs, max_log)
      ipre_llrs[:] = (ipost_llrs[self.turbo_deint]- ipre_llrs[self.turbo_deint]- sys_llrs_bot[self.turbo_deint])

    # Final post-decoding LLRs and hard decisions
    post_info_bit_llrs = ipost_llrs[self.turbo_deint]
    info_bits_hat = (post_info_bit_llrs < 0).astype(int)

    return info_bits_hat, post_info_bit_llrs

In [51]:
if __name__=="__main__":
  tc=Turbo(13,[11])
  
  def output(EbNodB):

    #prepare some constants
    MAX_ERR=8
    count_ball=0
    count_berr=0
    count_all=0
    count_err=0

    K=1000
    #count_berr_mat=np.zeros(1000)

    while count_err<MAX_ERR:
    #print("\r"+str(count_err),end="")

      #main calcuration
      information=np.random.randint(0,2,K)
      codeword=tc.encode(information)
      Lc=-1*ch.generate_LLR(codeword,EbNodB)
      #Lc[::3]=lambda_p1
      #Lc[1::3]=lambda_p2
      
      EST_information,_=tc.decode(Lc,18)
      #information,EST_information=tc.main_func(EbNodB)
      
      #calculate block error rate
      if np.any(information!=EST_information):
          count_err+=1
      count_all+=1

      #calculate bit error rate 
      count_berr+=np.sum(information!=EST_information)
      #count_berr_mat+=(information!=EST_information)
      #print(count_berr_mat)
      count_ball+=len(information)

      print("\r","count_all=",count_all,",count_err=",count_err,"count_ball=",count_ball,"count_berr=",count_berr,end="")

    return count_err,count_all,count_berr,count_ball

  EbNodB_range=np.array([-4,-3,-2,-1,0,1,2])
  for i in EbNodB_range:
    print(i)
    _,_,a,b=output(i)
    print(a/b)

-4
 count_all= 8 ,count_err= 8 count_ball= 8000 count_berr= 40500.50625
-3
 count_all= 3 ,count_err= 3 count_ball= 3000 count_berr= 1495